In [1]:
# in this notebook, we watch the batch and try fancy tf and torch functions

In [2]:
import pickle
import torch
import gym
from lib import wrappers
from lib import tf_dqn_model
from lib import dqN_model2

In [3]:
batch = None
with open('batch.pkl', 'rb') as f:
    batch = pickle.load(f)
GAMMA = 0.99

In [4]:
states, actions, rewards, dones, next_states = batch
device="cpu"

In [5]:
env = wrappers.make_env("PongNoFrameskip-v4")
net =dqN_model2.DQN(env.observation_space.shape,env.action_space.n).to(device)
tf_net = tf_dqn_model.DQN(env.observation_space.shape,env.action_space.n)
tf_tgt_net = tf_dqn_model.DQN(env.observation_space.shape,env.action_space.n)

In [6]:
states_v = torch.tensor(states).to(device)
actions_v = torch.tensor(actions).to(device)
rewards_v = torch.tensor(rewards).to(device)
done_mask = torch.BoolTensor(dones).to(device)
next_states_v = torch.tensor(next_states).to(device)
state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)

In [7]:
net(states_v).shape

torch.Size([32, 6])

In [8]:
actions_v.shape

torch.Size([32])

In [9]:
actions_v.unsqueeze(-1).shape

torch.Size([32, 1])

In [10]:
net(states_v).gather(1, actions_v.unsqueeze(-1)).shape

torch.Size([32, 1])

In [11]:
state_action_values.shape

torch.Size([32])

In [12]:
import tensorflow as tf
tf.executing_eagerly()


True

In [13]:
states, actions, rewards, dones, next_states = batch
states_v = tf.convert_to_tensor(states, tf.float32)
actions_v = tf.convert_to_tensor(actions, tf.int64)
rewards_v = tf.convert_to_tensor(rewards, tf.float32)
done_mask = tf.convert_to_tensor(dones, dtype=tf.bool)
done_mask = tf.expand_dims(tf.cast(done_mask, dtype=tf.float32), axis=1)
next_states_v = tf.convert_to_tensor(next_states, tf.float32)
#state_action_values = tf.squeeze( tf.gather_nd(net(states_v),tf.expand_dims(actions_v, -1)), -1)

In [14]:
def torch_gather(x, indices, gather_axis):
    # if pytorch gather indices are
    # [[[0, 10, 20], [0, 10, 20], [0, 10, 20]],
    #  [[0, 10, 20], [0, 10, 20], [0, 10, 20]]]
    # tf nd_gather needs to be
    # [[0,0,0], [0,0,10], [0,0,20], [0,1,0], [0,1,10], [0,1,20], [0,2,0], [0,2,10], [0,2,20],
    #  [1,0,0], [1,0,10], [1,0,20], [1,1,0], [1,1,10], [1,1,20], [1,2,0], [1,2,10], [1,2,20]]

    # create a tensor containing indices of each element
    all_indices = tf.where(tf.fill(indices.shape, True))
    gather_locations = tf.reshape(indices, [indices.shape.num_elements()])

    # splice in our pytorch style index at the correct axis
    gather_indices = []
    for axis in range(len(indices.shape)):
        if axis == gather_axis:
            gather_indices.append(gather_locations)
        else:
            gather_indices.append(all_indices[:, axis])

    gather_indices = tf.stack(gather_indices, axis=-1)
    gathered = tf.gather_nd(x, gather_indices)
    reshaped = tf.reshape(gathered, indices.shape)
    return reshaped

In [15]:
@tf.function
def train_step(batch):
    with tf.GradientTape() as tape:
        states, actions, rewards, dones, next_states = batch
        states_v = tf.convert_to_tensor(states, tf.float32)
        actions_v = tf.convert_to_tensor(actions, tf.int64)
        rewards_v = tf.convert_to_tensor(rewards, tf.float32)
        done_mask = tf.convert_to_tensor(dones, dtype=tf.uint8)
        done_mask = tf.cast(done_mask, dtype=tf.float32)
        next_states_v = tf.convert_to_tensor(next_states, tf.float32)
        state_action_values = tf.squeeze(torch_gather(tf_net(states_v),tf.expand_dims(actions_v, -1),1), -1)
        #print("state_action_values",state_action_values.shape)
        #print("tf.math.reduce_max(tgt_net(next_states_v),1,False)", tf.math.reduce_max(tgt_net(next_states_v),1).shape)
        next_states_values =  tf.math.reduce_max(tf_tgt_net(next_states_v),1) * done_mask
        #print("done_mask", done_mask.shape)
        #print("next_states_values",next_states_values.shape)
        print("next_states_values",next_states_values.shape)
        print("rewards_v", rewards_v.shape)
        expected_state_action_values = rewards_v + next_states_values * GAMMA
        #print("expected_state_action_values",expected_state_action_values.shape)
#         loss = loss_object(state_action_values, expected_state_action_values)
#         gradients = tape.gradient(loss, net.trainable_variables)
#         optimizer.apply_gradients(zip(gradients, net.trainable_variables))

In [16]:
train_step(batch)

next_states_values (32,)
rewards_v (32,)
next_states_values (32,)
rewards_v (32,)


In [21]:
states, actions, rewards, dones, next_states = batch

In [29]:
import numpy as np

In [34]:
~np.asarray(dones,np.bool)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [23]:
~1

-2